# Week 6 - Production and Scaling

## Objectives

- Understand context management at scale
- Create a stateful user interface for multiple people to interact with a chat bot




## Chainlit

It's time to see your chat in a more familiar context. To get a ChatGPT-like interface to interact with out model, we will use **chainlit**. Its installation is simple.

Go to the terminal and run the following:

```cmd
pip install chainlit
chainlit hello
```

Then you should be set up!

# Context Management

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulation Variables
input_tokens_per_prompt = 500
output_tokens_per_prompt = 200
prompts_per_day_per_user = 10
num_users = 50
cost_per_1m_input = 0.50  # in dollars
cost_per_1m_output = 1.50 # in dollars
simulation_days = 365

# Calculate the base cost factor used in both formulas
# Formula term: (i*ic + o*oc) / 1,000,000
# i=input, o=output, ic=input_cost, oc=output_cost
base_cost_factor = (input_tokens_per_prompt * cost_per_1m_input + 
                    output_tokens_per_prompt * cost_per_1m_output) / 1_000_000

# Graph 2 Data: Cost per prompt (Single User)
# Formula: p * (i*ic + o*oc) / 1,000,000
total_prompts = simulation_days * prompts_per_day_per_user
prompt_indices = np.arange(1, total_prompts + 1)
cost_per_prompt_single_user = prompt_indices * base_cost_factor

# Graph 1 Data: Days vs Money Spent (Cumulative for all users)
# Formula: p*(p+1)/2 * (i*ic + o*oc) / 1,000,000
days = np.arange(1, simulation_days + 1)
prompts_cumulative = days * prompts_per_day_per_user
cumulative_cost_single_user = (prompts_cumulative * (prompts_cumulative + 1) / 2) * base_cost_factor
daily_costs = cumulative_cost_single_user * num_users

# Graph 1: Days vs Money Spent (Cumulative)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(days, daily_costs, marker='o', color='b')
plt.title(f'Cumulative Cost over {simulation_days} Days ({num_users} Users)')
plt.xlabel('Days')
plt.ylabel('Total Money Spent ($)')
plt.grid(True)

# Graph 2: Number of Prompts vs Cost per Prompt (Single User)
plt.subplot(1, 2, 2)
plt.plot(prompt_indices, cost_per_prompt_single_user, color='r')
plt.title('Cost per Prompt (Single User Context Growth)')
plt.xlabel('Prompt Number')
plt.ylabel('Cost per Prompt ($)')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Define the cap for context growth
context_growth_cap = 50

# Recalculate Cost per prompt (Single User) with the cap
# Formula: min(p, 50) * base_cost_factor
# We use np.minimum to apply the cap element-wise to the array of prompt indices
capped_prompt_indices = np.minimum(prompt_indices, context_growth_cap)
cost_per_prompt_single_user_capped = capped_prompt_indices * base_cost_factor

# Recalculate Cumulative Cost (Single User) with the cap
# We sum up the costs of individual prompts cumulatively
cumulative_cost_single_user_capped = np.cumsum(cost_per_prompt_single_user_capped)

# Recalculate Daily Costs for all users
# We need to map the cumulative cost at the end of each day
# Since prompts_per_day_per_user is 10, day 1 ends at prompt index 10, day 2 at 20, etc.
indices_at_day_end = (days * prompts_per_day_per_user) - 1 # -1 for 0-based indexing if needed, but cumulative_cost array is 1-based length
daily_cumulative_costs_single = cumulative_cost_single_user_capped[indices_at_day_end]
daily_costs_capped = daily_cumulative_costs_single * num_users

# Plotting the new capped results
plt.figure(figsize=(12, 5))

# Graph 1: Days vs Money Spent (Cumulative) with Cap
plt.subplot(1, 2, 1)
plt.plot(days, daily_costs_capped, marker='o', color='b')
plt.title(f'Cumulative Cost (Capped at {context_growth_cap} prompts context)')
plt.xlabel('Days')
plt.ylabel('Total Money Spent ($)')
plt.grid(True)

# Graph 2: Number of Prompts vs Cost per Prompt (Single User) with Cap
plt.subplot(1, 2, 2)
plt.plot(prompt_indices, cost_per_prompt_single_user_capped, color='r')
plt.title(f'Cost per Prompt (Capped at {context_growth_cap})')
plt.xlabel('Prompt Number')
plt.ylabel('Cost per Prompt ($)')
plt.grid(True)

plt.tight_layout()
plt.show()